# Importing Necessary libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import Sequential,utils
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

# Data Loading

In [ ]:
!ls ../input/heartbeat/mitbih_test.csv  mitbih_train.csv  ptbdb_abnormal.csv  ptbdb_normal.csv

# Importing Train Data

In [ ]:
data = pd.read_csv("../input/heartbeat/mitbih_train.csv", header=None)
data=pd.DataFrame(data)

In [ ]:
data

# Data Preprocessing

In [ ]:
data.isnull().sum()

In [ ]:
data.describe().T

In [ ]:
data[187].value_counts()

As we can see our data is imbalanced so we will apply Undersampling

In [ ]:
# Applying Undersampling
class_1 = data[data[187]==1.0]
class_2 = data[data[187]==2.0]
class_3 = data[data[187]==3.0]
class_4 = data[data[187]==4.0]
class_0 = data[data[187]==0.0].sample(n = 8000)
data = pd.concat([class_1, class_2, class_3, class_4, class_0]).sample(frac=1)

In [ ]:
data[187].value_counts()

# Spliting Data into Train and Test and Reshaping the data

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(data.drop([187], axis=1), data[187], test_size = 0.1)

In [ ]:
xtrain = np.array(xtrain).reshape(xtrain.shape[0], xtrain.shape[1], 1)
xtest = np.array(xtest).reshape(xtest.shape[0], xtest.shape[1], 1)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(25,2))
ax.plot(data[data[187]==float(1)].sample(1).iloc[0,:186])


#Applying CNN

In [ ]:
neu = Sequential()

neu.add(Conv1D(filters=32, kernel_size=(3,), padding='same', activation='relu', input_shape = (xtrain.shape[1],1)))
neu.add(Conv1D(filters=64, kernel_size=(3,), padding='same', activation='relu')) 
neu.add(Conv1D(filters=128, kernel_size=(5,), padding='same', activation='relu'))    

neu.add(MaxPool1D(pool_size=(3,), strides=2, padding='same'))
neu.add(Dropout(0.5))

neu.add(Flatten())

neu.add(Dense(units = 512, activation='relu'))
neu.add(Dense(units = 1024, activation='relu'))
neu.add(Dense(units = 5, activation='softmax'))

neu.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
his = neu.fit(xtrain, ytrain, epochs = 10)

# Making Predictions using training data

In [ ]:
ypred_train = neu.predict(xtest)

In [ ]:
y_lbl = [np.where(i == np.max(i))[0][0] for i in ypred_train]
mat = confusion_matrix(ytest, y_lbl)
fig, ax = plt.subplots(figsize=(7,7))
sns.heatmap(mat, annot = True)

In [ ]:
print("Accuracy score of the predictions: {0}".format(accuracy_score(y_lbl, ytest)))

# Loading Test Data Test

In [ ]:
test = pd.read_csv("../input/heartbeat/mitbih_test.csv", header=None)
test=pd.DataFrame(test)

In [ ]:
test

# Data Preprocessing

In [ ]:
test[187].value_counts()

In [ ]:
# Applying Under Sampling
class_1 = test[test[187]==1.0]
class_2 = test[test[187]==2.0]
class_3 = test[test[187]==3.0]
class_4 = test[test[187]==4.0]
class_0 = test[test[187]==0.0].sample(n = 3000)
test = pd.concat([class_1, class_2, class_3, class_4, class_0]).sample(frac=1)

In [ ]:
test[187].value_counts()

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(25,2))
ax.plot(test[test[187]==float(1)].sample(1).iloc[0,:186])


# Making Predictions using testing data 

In [ ]:
xtest = test.drop([187], axis=1) 
ytest = test[187]

xtest = np.array(xtest).reshape(xtest.shape[0], xtest.shape[1], 1)

In [ ]:
ytest_pred = neu.predict(xtest)

In [ ]:
ytest_lbl = [np.where(i == np.max(i))[0][0] for i in ytest_pred]
mat = confusion_matrix(ytest, ytest_lbl)
fig, ax = plt.subplots(figsize=(7,7))
sns.heatmap(mat, annot = True)

In [ ]:
print("Accuracy score of the predictions: {0}".format(accuracy_score(ytest_lbl, ytest)))